In [1]:
from paje.base.data import Data
from paje.activelearning.strategy.agnostic.random import RandomSampling
from sklearn import datasets
from sklearn.model_selection import train_test_split
from paje.activelearning.datasets import skldatasets
import pandas as pd
import random as rd
import numpy as np


In [2]:
unlabeled_data, labeled_data, unlabeled_iris = skldatasets.iris_dataset()

In [3]:
unlabeled_iris.X.dtype

dtype('float64')

In [4]:
rdss = RandomSampling(unlabeled_iris, 'paje_data')

In [7]:
rdss.query(2).X

array([[7.6, 3. , 6.6, 2.1],
       [6.7, 3.1, 5.6, 2.4]])